# `withdrawal` data

The withdrawal data is defined at the rally level for each category.

In [1]:
# Load in the required pandas package
import pandas as pd

# Generate the API URL pattern
dakar_api_template = "https://www.dakar.live.worldrallyraidchampionship.com/api/{path}"

# Define the year
YEAR = 2025
# Define the category
CATEGORY = "A"

# Define the API path to the withdrawal resource
# Use a Python f-string to instantiate variable values directly
withdrawal_path = f"withdrawal-{YEAR}-{CATEGORY}"

# Define the URL
withdrawal_url = dakar_api_template.format(path=withdrawal_path)

# Preview the path and the URL
withdrawal_path, withdrawal_url

('withdrawal-2025-A',
 'https://www.dakar.live.worldrallyraidchampionship.com/api/withdrawal-2025-A')

Within the withrdrawal data, we get a list of withdrawals by stage.

In [4]:
withdrawal_df = pd.read_json(withdrawal_url)

# Use the stage number as the index
withdrawal_df.set_index("stage", drop=False, inplace=True)
withdrawal_df

,_id,_bind,_updatedAt,_parent,list,stage
stage,,,,,,
4,withdrawal-2025-A-4,withdrawal-2025-A,1736346802326,stage-2025-A:8002af597f2d091dc91450266fab52d6,"[{'bib': 219, 'reason': '2', 'team': {'bib': 2...",4
3,withdrawal-2025-A-3,withdrawal-2025-A,1736346740019,stage-2025-A:127ed31c3eff071ba6a7e88fe01083ef,"[{'bib': 225, 'reason': '1', 'team': {'bib': 2...",3
2,withdrawal-2025-A-2,withdrawal-2025-A,1736346664755,stage-2025-A:9afdc50df69247c38b23cef4ed50bd14,"[{'bib': 223, 'reason': '1', 'team': {'bib': 2...",2


In [9]:
withdrawals_by_stage = withdrawal_df["list"].explode()
withdrawals_by_stage_index = withdrawals_by_stage.index

withdrawals_by_stage_df = pd.json_normalize(withdrawals_by_stage)

withdrawals_by_stage_df["stage"] = withdrawals_by_stage_index
withdrawals_by_stage_df.head()

,bib,reason,team.bib,team.brand,team.model,team.vehicle,team.vehicleImg,team.clazz,team.w2rc,team.competitors,stage
0,219,2,219,DACIA,SANDRIDER,THE DACIA SANDRIDERS,https://img.aso.fr/core_app/img-motorSports-da...,96c0869600e0013dbf5f86f60e5c4da4,True,"[{'name': 'S. LOEB', 'firstName': 'SEBASTIEN',...",4
1,236,1,236,MD,OPTIMUS,MD RALLYE SPORT,https://img.aso.fr/core_app/img-motorSports-da...,f00d7ec8d2d96e9cf11aa515109376cf,False,"[{'name': 'E. AMOS', 'firstName': 'EUGENIO', '...",4
2,258,1,258,RED-LINED,NAVARRA VK 56,WALCHER RACING TEAM,https://img.aso.fr/core_app/img-motorSports-da...,f666973e89db183ecfefc75c3af8ffb1,False,"[{'name': 'M. WALCHER', 'firstName': 'MARKUS',...",4
3,337,1,337,BRP,CAN-AM MAVERICK X3,COLORADO MOTORSPORT,NaN,a0a6386a4b9a61b73b036a50966345c0,False,"[{'name': 'C. LUMSDEN', 'firstName': 'CRAIG', ...",4
4,408,1,408,BRP,CAN-AM MAVERICK R,SOUTH RACING CAN-AM,https://img.aso.fr/core_app/img-motorSports-da...,9a68ed3c41c5c7a1642df5d93458baa6,True,"[{'name': 'D. MARTINEZ', 'firstName': 'DIEGO',...",4


There seems to be a `reason` code explaining the reason for the withdrawal, but I haven't (yet!) found a metadata feed the explains those code values.